In [ ]:
import cv2
import numpy as np
impo
from scipy.fftpack import dct, idct
from pathlib import Path
import os, sys


In [ ]:
def dct2(b):
    return dct(dct(b.T, norm='ortho').T, norm='ortho')

def idct2(b):
    return idct(idct(b.T, norm='ortho').T, norm='ortho')

def psnr(a, b):
    mse = np.mean((a.astype(np.float32) - b.astype(np.float32))**2)
    return 99 if mse == 0 else 10*np.log10(255*255/mse)

def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)

# JPEG Table (교재 Table 9‑1)
QY = np.array([
 [16,11,10,16,24,40,51,61],
 [12,12,14,19,26,58,60,55],
 [14,13,16,24,40,57,69,56],
 [14,17,22,29,51,87,80,62],
 [18,22,37,56,68,109,103,77],
 [24,35,55,64,81,104,113,92],
 [49,64,78,87,103,121,120,101],
 [72,92,95,98,112,100,103,99]
], dtype=np.float32)

def main(img_path='Lena.bmp'):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print('이미지 못 읽음'); return

    H, W = img.shape
    # 8의 배수로 패딩 (그냥 0 패딩)
    H2, W2 = (H+7)//8*8, (W+7)//8*8
    pad = np.zeros((H2, W2), np.uint8)
    pad[:H, :W] = img
    img = pad
    H, W = H2, W2

    #  블록 함수 
    def run_block(fn):
        out = np.zeros_like(img, np.float32)
        for y in range(0, H, 8):
            for x in range(0, W, 8):
                blk = img[y:y+8, x:x+8].astype(np.float32) - 128
                out[y:y+8, x:x+8] = fn(blk) + 128
        return np.clip(out, 0, 255).astype(np.uint8)

    for n in range(2, 7):
        def keep_n(b):
            D = dct2(b)
            D[n:, :] = 0
            D[:, n:] = 0
            return idct2(D)
        recon = run_block(keep_n)
        cv2.imwrite(f'recon_{n}x{n}.png', recon)
        print(f'{n}×{n} PSNR = {psnr(img, recon):.2f} dB')

    # JPEG 양자화 테스트
    def jpeg_like(b):
        D = dct2(b)
        Q = np.round(D / QY)   
        Dq = Q * QY           
        return idct2(Dq)
    jpeg_recon = run_block(jpeg_like)
    cv2.imwrite('jpeg_recon.png', jpeg_recon)
    print(f'JPEG‑table PSNR = {psnr(img, jpeg_recon):.2f} dB')
    show('orig', img); show('jpeg', jpeg_recon)

if __name__ == '__main__':
    if len(sys.argv)==2:
        main(sys.argv[1])
    else:
        main()


이미지 못 읽음
